In [1]:
import os
import yaml
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
with open('./package.yaml') as f:
    file = yaml.load(f, Loader=yaml.FullLoader)
    POSEDATAPATH = file["path"]["pose_keypoints"]
    FACEDATAPATH = file["path"]["faceMesh_keypoints"]
    LABELPATH = file["path"]["labelPATH"]

In [3]:
label = pd.read_excel(LABELPATH)

In [4]:
label = label[["번호", "파일명", "한국어"]].sort_values(by="번호", ascending=True).iloc[:2000]   # 컬럼 인덱싱 && 정렬 && 행 2000개 가져오기
label

,번호,파일명,한국어
10460,1,KETI_SL_0000000001.MOV,0
10440,2,KETI_SL_0000000002.MOV,1
10420,3,KETI_SL_0000000003.MOV,2
10399,4,KETI_SL_0000000004.MOV,3
10379,5,KETI_SL_0000000005.MOV,4
...,...,...,...
804,1996,KETI_SL_0000001996.MOV,트랙터
784,1997,KETI_SL_0000001997.MOV,트럭
764,1998,KETI_SL_0000001998.MOV,파도
9703,1999,KETI_SL_0000001999.MOV,38


In [5]:
pose_datapack = []
face_datapack = []

for file in os.listdir(POSEDATAPATH):   # Pose Keypoints - "pose" 컬럼 추가
    r_path = os.path.join(POSEDATAPATH, file)
    data = np.load(r_path)
    pose_datapack.append(data)

# for file in os.listdir(FACEDATAPATH):   # FaceMesh keypoints - "face" 컬럼 추가
#     r_path = os.path.join(FACEDATAPATH, file)
#     data = np.load(r_path)
#     face_datapack.append(data)

In [6]:
# label["pose"] = pose_datapack
# label["face"] = face_datapack

In [7]:
# label['한국어'].nunique()   # 총 레이블 수 - 419

In [35]:
max_len = 0 # keypoints 최대 길이 변수

for idx, data in label.iterrows():
    keypoints_len = data['pose'].shape[0]  # example shape = (141, 33, 3)
    if keypoints_len >= max_len:
        max_len = keypoints_len

print(f"Max Length :", max_len)

padded_poses = []

for idx, data in label.iterrows():  # keypoints padding
    pose = data['pose']
    keypoints_len = pose.shape[0]
    
    pad_width = max_len - keypoints_len  # 필요한 padding값 계산
    
    if pad_width > 0:
        padding = ((0, pad_width), (0, 0), (0, 0))  
        padded_pose = np.pad(pose, padding, mode='constant', constant_values=0) # zero padding
    else:
        padded_pose = pose
    
    padded_poses.append(padded_pose)
pose_array = np.array(padded_poses)
print(f"pose_array Shape : {pose_array.shape}")

Max Length : 210
pose_array Shape : (2000, 210, 33, 3)


In [30]:
np.save('./pose_array.npy', pose_array)